In [1]:
%matplotlib inline

In [2]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np
from shapely.ops import nearest_points

In [13]:
import requests
def _osrm_tdm_request(coords):
    url_coords = ';'.join([str(coord.lon) + ',' + str(coord.lat) for coord in coords])
    url_server = 'http://0.0.0.0:5000/table/v1/driving/'
    url_options = 'fallback_speed=0&annotations=duration,distance'
    url_full = '{}{}?{}'.format(url_server, url_coords, url_options)
    resp = requests.get(url=url_full)

    jresp = resp.json()
    if jresp.get('code') != 'Ok':
        print(jresp.get('code'))
        print(jresp.get('message'))
        resp.raise_for_status()

    return jresp.get('durations'), jresp.get('distances')

class Coord(object):
    """Coordinate.

    Parameters
    ----------
    lat : <float> latitude
    lon : <float> longitude
    latlon : <list> list with both lat and long. Latitude first!
    """
    def __init__(self, lat=None, lon=None, latlon=None):
        if latlon is not None:
            if len(latlon) != 2:
                raise Exception("Wrong coordinate latlon format. Should be a list of two floats.")
            self.lat = latlon[0]
            self.lon = latlon[1]
        elif lat is None or lon is None:
            raise Exception("Coordinates not provided")
        else:
            self.lat = lat
            self.lon = lon

    def to_json(self):
        return json.dumps(self, default=lambda o: self._try(o), sort_keys=True, indent=4, separators=(',', ':'))

    @staticmethod
    def _try(o):
        try:
            if o.__class__ == Coord:
                raise Exception()
            return o.__dict__
        except:
            return str(o)

In [137]:
b1_sams = geopandas.read_file('data/GIS/B1_samsSW_20131231_shp/B1_sw_region.shp')
b1_sams = b1_sams.loc[(b1_sams.KOMMUN.astype(int) < 1300) & (b1_sams.KOMMUN.astype(int) >= 1200)]
b1_sams = b1_sams.to_crs(epsg=4326)

In [138]:
b1_sams.head()

,SAMSCODE,Samsnamn,KOMMUN,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44,Ald45_64,Ald65_W,TotBef,geometry
0,12800358,Tygelsj 2,1280,170,224,43,32,338,222,56,1085,"POLYGON ((12.94854461084082 55.52745131666308,..."
1,12800354,Vstra Klagstorp 1,1280,233,271,44,41,471,306,108,1474,"POLYGON ((12.93426940280717 55.53865509268457,..."
2,12800352,Bunkeflo 3,1280,11,16,6,3,39,30,9,114,"POLYGON ((12.93054342116049 55.54900936648132,..."
3,12800348,Bunkeflo 3,1280,159,104,11,10,312,95,23,714,"POLYGON ((12.93888202453416 55.5648631563455, ..."
4,12800347,Bunkeflo 3,1280,137,106,19,25,235,136,55,713,"POLYGON ((12.95479624389526 55.56082281754748,..."


In [139]:
b1_sams['SAMSCODE'] = b1_sams.SAMSCODE.astype(int)

In [140]:
largest_SAMS = b1_sams.groupby(['KOMMUN'], sort=False)['TotBef'].transform(max) == b1_sams.TotBef
df_komm = b1_sams[largest_SAMS][['KOMMUN', 'geometry']].set_index(['KOMMUN'])

In [141]:
#df_komm = geopandas.GeoDataFrame({'komm_codes': komm_codes}, geometry=komm_centroids).set_index('komm_codes')

In [142]:
df_komm['geometry'] = df_komm.centroid

In [9]:
stops = geopandas.read_file('data/GIS/stations/stations.shp')
stops.head()

,ID,DESIGNATIO,FunkBin,geometry
0,14001,A,100,POINT (13.09159803706982 55.90754136603621)
1,14001,B,100,POINT (13.09161680992804 55.9077304689475)
2,14017,A,100,POINT (12.954893814977 55.92540436675255)
3,14020,A,100,POINT (13.10660175482446 55.90869801283589)
4,14020,B,100,POINT (13.10638338262923 55.90838772802925)


In [143]:
df_komm['geometry'] = [stops.loc[stops.distance(geom).idxmin()].geometry for geom in df_komm.geometry]

In [144]:
df_komm['x'] = df_komm.geometry.apply(lambda x: x.x)
df_komm['y'] = df_komm.geometry.apply(lambda x: x.y)
xy = df_komm.loc[:, ['x', 'y']]
# this point is a large train station without car access
xy.loc['1270']['x'] = 13.959357557127321
result = [[(xy.loc[row], xy.loc[col]) for col in xy.index]
          for row in xy.index]
coord_matrix = pd.DataFrame(result, index=xy.index, columns=xy.index)

In [152]:
a = pd.DataFrame(coord_matrix.unstack(level=-1))
a.index.names = ['to','from']
a = a.reset_index()
a['from_x'] = [c[0].y for c in a[0]]
a['from_y'] = [c[0].x for c in a[0]]
a['to_x'] = [c[1].y for c in a[0]]
a['to_y'] = [c[1].x for c in a[0]]
a.head()

,to,from,0,from_x,from_y,to_x,to_y
0,1233,1233,"([12.953316686080452, 55.41874632495786], [12....",55.418746,12.953317,55.418746,12.953317
1,1233,1282,"([12.900650620450076, 55.85961827103177], [12....",55.859618,12.900651,55.418746,12.953317
2,1233,1284,"([12.569797702858919, 56.151491879333555], [12...",56.151492,12.569798,55.418746,12.953317
3,1233,1292,"([12.862799252520684, 56.24662120132786], [12....",56.246621,12.862799,55.418746,12.953317
4,1233,1283,"([12.692785367827911, 56.08003333279364], [12....",56.080033,12.692785,55.418746,12.953317


In [146]:
a.to_csv('data/distance_matix_municipalities.csv', index=False, columns=['from', 'from_x', 'from_y', 'to', 'to_x', 'to_y'], header=False)

In [153]:
xy['coord'] = [Coord(lon=lon, lat=lat) for _, lon, lat in xy.itertuples()]
j = np.array(_osrm_tdm_request(xy.coord.values))

In [157]:
durations = j[0]
distances = j[1]
duration_matrix = pd.DataFrame(durations, index=xy.index, columns=xy.index)
distance_matrix = pd.DataFrame(distances, index=xy.index, columns=xy.index)

In [158]:
distance_matrix.to_csv('data/kommun_distance_matrix.csv', sep=';', index=True)

In [159]:
distance_matrix

KOMMUN,1233,1282,1284,1292,1283,1277,1287,1263,1281,1214,...,1270,1293,1256,1290,1272,1291,1273,1286,1266,1265
KOMMUN,,,,,,,,,,,,,,,,,,,,,
1233,0.0,61210.9,102716.7,109949.1,92484.7,100371.5,16222.9,33427.8,43987.1,70967.7,...,87426.5,115584.5,136448.2,124269.9,140678.4,115060.2,166034.1,70612.7,76023.1,70755.3
1282,61742.7,0.0,43920.0,51152.4,33688.0,41574.8,70769.9,52816.6,29871.4,20781.5,...,89032.3,97271.3,114460.8,102282.5,118691.0,115073.3,147361.9,83991.2,54035.7,70512.1
1284,103779.5,43871.3,0.0,25753.8,14319.3,29692.5,112806.7,94853.3,71908.2,49228.3,...,131069.1,85389.0,108273.6,123746.9,140155.5,157110.1,135479.6,126028.0,105493.9,112548.9
1292,111962.1,52053.9,25554.5,0.0,29939.2,19865.9,120989.3,103035.9,80090.7,51167.2,...,139251.7,71606.2,95859.3,111332.6,127741.1,165292.7,114205.2,134210.6,83310.6,120731.5
1283,93445.2,33537.0,14804.5,28819.4,0.0,19358.2,102472.4,84519.1,61573.9,38894.0,...,120734.8,75054.8,97939.3,113412.7,129821.2,146775.8,125145.4,115693.7,95159.6,102214.6
1277,101357.3,41449.1,29622.1,18872.0,19390.1,0.0,110384.5,92431.2,69486.0,26473.3,...,128646.9,57024.0,79908.6,95381.9,111790.4,154687.9,108640.2,123605.8,66638.5,110126.7
1287,16220.0,70183.6,111689.4,118921.8,101457.4,109344.2,0.0,18266.7,52959.8,79940.4,...,72933.2,124557.3,145420.9,133242.6,149651.2,100566.9,175006.8,56119.5,84995.8,79728.0
1263,34862.8,52337.2,93843.0,101075.4,83611.0,91497.8,18280.3,0.0,35113.4,62094.0,...,56345.0,106710.8,127574.5,115396.2,131804.7,83978.8,157160.4,39531.3,67149.4,51957.8
1281,44882.2,29334.3,70840.1,78072.5,60608.1,68494.8,53909.5,35956.1,0.0,33816.4,...,57890.9,73786.9,94650.6,82472.3,98880.8,83931.9,124236.5,52849.8,34225.5,39370.8


# SAMS zones

In [4]:
b1_sams = geopandas.read_file('data/GIS/B1_samsSW_20131231_shp/B1_sw_region.shp')
b1_sams = b1_sams.loc[(b1_sams.KOMMUN.astype(int) < 1300) & (b1_sams.KOMMUN.astype(int) >= 1200)]
b1_sams = b1_sams.to_crs(epsg=4326)
b1_sams['geometry'] = b1_sams.centroid

/home/ai6644/anaconda3/envs/jup/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [5]:
stops = geopandas.read_file('data/GIS/stations/stations.shp')

In [6]:
target_zones = b1_sams

In [7]:
target_zones['geometry'] = [stops.loc[stops.distance(geom).idxmin()].geometry for geom in target_zones.geometry]
target_zones = target_zones.set_index('SAMSCODE')

In [8]:
target_zones['x'] = target_zones.geometry.apply(lambda x: x.x)
target_zones['y'] = target_zones.geometry.apply(lambda x: x.y)
# other_zones['x'] = other_zones.geometry.apply(lambda x: x.x)
# other_zones['y'] = other_zones.geometry.apply(lambda x: x.y)

# xy = pd.concat([target_zones.loc[:, ['x', 'y']], other_zones.loc[:, ['x', 'y']]])
xy = target_zones.loc[:, ['x', 'y']]

result = [[(xy.loc[row], xy.loc[col]) for col in xy.index]
          for row in xy.index]
sams_coord_matrix = pd.DataFrame(result, index=xy.index, columns=xy.index)

In [30]:
xy.head()

,x,y
SAMSCODE,,
12800358,12.932115,55.523693
12800354,12.900639,55.523489
12800352,12.941658,55.529479
12800348,12.934851,55.556125
12800347,12.962809,55.557756


In [53]:
xy = xy.loc[:,['x','y']]

In [26]:
xy['coord'] = [Coord(lon=lon, lat=lat) for _, lon, lat in xy.itertuples()]
j = np.array(_osrm_tdm_request(xy.coord.values))

InvalidOptions
fallback_speed must be > 0


HTTPError: 400 Client Error: Bad Request for url: http://0.0.0.0:5000/table/v1/driving/12.932114669477745,55.52369280508821;12.900638850870497,55.523488909117624;12.941657520186277,55.52947883848362;12.934850939770925,55.55612472062334;12.962809342870925,55.55775649799227;12.91871922166994,55.54371308362601;12.920251705535076,55.55437074462042;12.917617016697964,55.55009415991655;12.929696376990865,55.55485354625889;12.917617016697964,55.55009415991655;12.918421322481057,55.555118370215396;12.962613048233251,55.56595609780831;12.954993827791482,55.56825011052613;12.962613048233251,55.56595609780831;12.920685465535442,55.40542666185484;12.9439752842356,55.414007924402085;12.927378027264595,55.406779652605785;12.953316686080452,55.41874632495786;12.851360629374822,55.39411878724782;12.920685465535442,55.40542666185484;12.84325007911985,55.411769479155616;12.768689730964073,55.96250175667375;12.806616778239402,55.98496171268515;12.768793511838068,55.958316806227764;12.811473122083386,55.9

In [42]:
import polyline

In [69]:
poly = 'polyline(' + polyline.encode([[lat,lon] for _,lon, lat in xy.itertuples()]) + ')'

In [70]:
url_server = 'http://0.0.0.0:5000/table/v1/driving/'
url_options = 'fallback_speed=99999999999999999&annotations=duration,distance'
url_full = '{}{}?{}'.format(url_server, poly, url_options)
resp = requests.get(url=url_full)
jresp = resp.json()

In [74]:
distances = jresp.get('distances')
durations = jresp.get('durations')

In [75]:
duration_matrix = pd.DataFrame(durations, index=xy.index, columns=xy.index)
distance_matrix = pd.DataFrame(distances, index=xy.index, columns=xy.index)

In [77]:
distance_matrix

SAMSCODE,12800358,12800354,12800352,12800348,12800347,12800346,12800345,12800344,12800343,12800342,...,12650006,12650005,12650004,12650003,12650002,12650001,12640016,12640012,12640011,12640010
SAMSCODE,,,,,,,,,,,,,,,,,,,,,
12800358,0.0,3081.8,1140.1,6203.6,8048.1,3876.1,5192.5,4593.4,5834.9,4593.4,...,63866.0,58972.8,57936.0,61974.7,55859.3,55859.3,48791.7,48742.1,55912.9,57970.5
12800354,3081.8,0.0,2917.4,7981.0,9825.4,5653.4,6969.9,6370.8,7612.3,6370.8,...,65643.4,60750.2,59713.4,63752.0,57636.6,57636.6,50569.0,50519.5,57690.3,59747.9
12800352,1237.0,3016.3,0.0,5063.5,6908.0,2736.0,4052.4,3453.3,4694.8,3453.3,...,62725.9,57832.7,56795.9,60834.6,54719.2,54719.2,47651.6,47602.0,54772.8,56830.4
12800348,6490.9,8270.2,5411.7,0.0,1844.5,2725.4,1386.8,2008.3,763.0,2008.3,...,62157.1,57263.9,56227.1,60265.7,54150.3,54150.3,47082.7,47033.2,54203.9,56261.6
12800347,7945.1,9724.3,6865.9,1986.4,0.0,4179.5,2841.0,3462.5,2217.2,3462.5,...,60748.4,55855.2,54818.3,58857.0,52741.6,52741.6,45674.0,45624.4,52795.2,54852.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12650001,57012.8,58792.1,55933.6,54552.6,54002.8,55258.8,55407.2,56028.6,54783.3,56028.6,...,10118.7,5225.5,4188.6,8227.3,0.0,0.0,23322.9,27073.7,41320.0,37697.9
12640016,49939.9,51719.2,48860.7,47479.7,46929.9,48185.9,48334.2,48955.7,47710.4,48955.7,...,17848.0,24272.0,23235.2,25269.6,21360.6,21360.6,0.0,4617.0,13023.7,16880.0
12640012,49928.2,51707.5,48849.0,47468.0,46918.2,48174.2,48322.5,48944.0,47698.7,48944.0,...,17313.4,28022.7,26985.9,30332.2,25111.4,25111.4,4617.0,0.0,8406.8,9127.3


In [ ]:
distance_matrix.to_csv('data/sams_distance_matrix.csv', sep=';', index=True)

In [41]:
# pip install polyline

Note: you may need to restart the kernel to use updated packages.


# Switch on OTP to run this

In [148]:
import requests
multipart_form_data = {'scriptfile': ('OTP_travel_matrix.py', open('OTP_travel_matrix.py', 'rb'))}
r = requests.post(url='http://localhost:8080/otp/scripting/run', files=multipart_form_data)

In [150]:
tdm = pd.read_csv('data/time_distance_matrix_otp.csv', header=-1)
tdm.columns = ['from','to','duration', 'distance']
tdm = tdm.set_index(['from', 'to'])
tdm.head()

,,duration,distance
from,to,,
12650008,12650008,0,0.000
12700007,12650008,2502,29505.965
12700006,12650008,2334,28373.358
12700005,12650008,2440,28891.197
12700002,12650008,2431,30532.549


In [97]:
municipality_codes = tdm.index.levels[0].values
result = [[tdm.loc[(row, col)]['distance'] for col in municipality_codes]
          for row in municipality_codes]
distance_matrix = pd.DataFrame(result, index=municipality_codes, columns=municipality_codes, )

In [101]:
distance_matrix.to_csv('data/kommun_distance_matrix.csv', sep=';', index=True)

# The same for SAMS zones

In [15]:
b1_sams = geopandas.read_file('data/GIS/B1_samsSW_20131231_shp/B1_sw_region.shp')
b1_sams = b1_sams.loc[(b1_sams.KOMMUN.astype(int) < 1300) & (b1_sams.KOMMUN.astype(int) >= 1200)]
b1_sams = b1_sams.to_crs(epsg=4326)
b1_sams['geometry'] = b1_sams.centroid

In [16]:
stops = geopandas.read_file('data/GIS/stations/stations.shp')

In [114]:
# target_zones = b1_sams.loc[(b1_sams.KOMMUN == '1265') | (b1_sams.KOMMUN == '1270')]
target_zones = b1_sams

In [115]:
target_zones['geometry'] = [stops.loc[stops.distance(geom).idxmin()].geometry for geom in target_zones.geometry]

/home/ai6644/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
target_zones = target_zones.set_index('SAMSCODE')

In [52]:
# other_municipalities = b1_sams.loc[(b1_sams.KOMMUN != '1265') & (b1_sams.KOMMUN != '1270')]

In [53]:
# largest_SAMS = other_municipalities.groupby(['KOMMUN'], sort=False)['TotBef'].transform(max) == other_municipalities.TotBef
# other_zones = other_municipalities[largest_SAMS][['KOMMUN', 'geometry']].set_index(['KOMMUN'])

In [57]:
# other_zones['geometry'] = [stops.loc[stops.distance(geom).idxmin()].geometry for geom in other_zones.geometry]

In [31]:
target_zones['x'] = target_zones.geometry.apply(lambda x: x.x)
target_zones['y'] = target_zones.geometry.apply(lambda x: x.y)
# other_zones['x'] = other_zones.geometry.apply(lambda x: x.x)
# other_zones['y'] = other_zones.geometry.apply(lambda x: x.y)

# xy = pd.concat([target_zones.loc[:, ['x', 'y']], other_zones.loc[:, ['x', 'y']]])
xy = target_zones.loc[:, ['x', 'y']]

result = [[(xy.loc[row], xy.loc[col]) for col in xy.index]
          for row in xy.index]
sams_coord_matrix = pd.DataFrame(result, index=xy.index, columns=xy.index)

In [32]:
xy.head()

,x,y
SAMSCODE,,
12800358,12.932115,55.523693
12800354,12.900639,55.523489
12800352,12.941658,55.529479
12800348,12.934851,55.556125
12800347,12.962809,55.557756


In [33]:
a = pd.DataFrame(sams_coord_matrix.unstack(level=-1))
a.index.names = ['to','from']
a = a.reset_index()
a['from_x'] = [c[0].y for c in a[0]]
a['from_y'] = [c[0].x for c in a[0]]
a['to_x'] = [c[1].y for c in a[0]]
a['to_y'] = [c[1].x for c in a[0]]
a.head()

,to,from,0,from_x,from_y,to_x,to_y
0,12800358,12800358,"([12.932114669477745, 55.52369280508821], [12....",55.523693,12.932115,55.523693,12.932115
1,12800358,12800354,"([12.900638850870497, 55.523488909117624], [12...",55.523489,12.900639,55.523693,12.932115
2,12800358,12800352,"([12.941657520186277, 55.52947883848362], [12....",55.529479,12.941658,55.523693,12.932115
3,12800358,12800348,"([12.934850939770925, 55.55612472062334], [12....",55.556125,12.934851,55.523693,12.932115
4,12800358,12800347,"([12.962809342870925, 55.55775649799227], [12....",55.557756,12.962809,55.523693,12.932115


In [43]:
a.to_csv('data/distance_matrix_sams.csv', index=False, columns=['from', 'from_x', 'from_y', 'to', 'to_x', 'to_y'], header=False)

In [78]:
xy['coord'] = [Coord(lon=lon, lat=lat) for _, lon, lat in xy.itertuples()]
j = np.array(_osrm_tdm_request(xy.coord.values))

In [96]:
durations = j[0]
distances = j[1]
duration_matrix = pd.DataFrame(durations, index=xy.index, columns=xy.index)
distance_matrix = pd.DataFrame(distances, index=xy.index, columns=xy.index)

In [44]:
# import requests
# multipart_form_data = {'scriptfile': ('OTP_travel_matrix.py', open('OTP_travel_matrix.py', 'rb'))}
# r = requests.post(url='http://localhost:8080/otp/scripting/run', files=multipart_form_data)

In [45]:
# tdm = pd.read_csv('data/time_distance_matrix_otp.csv', header=-1)
# tdm.columns = ['from','to','duration', 'distance']
# tdm = tdm.set_index(['from', 'to'])
# tdm.head()

,,duration,distance
from,to,,
12650008,12650008,0,0.000
12700007,12650008,2502,29505.965
12700006,12650008,2334,28373.358
12700005,12650008,2440,28891.197
12700002,12650008,2431,30532.549


In [51]:
# codes = tdm.index.levels[0].values
# result = [[tdm.loc[(row, col)]['distance'] for col in codes]
#           for row in codes]
# distance_matrix = pd.DataFrame(result, index=codes, columns=codes, )

In [113]:
distance_matrix.to_csv('data/sams_distance_matrix.csv', sep=';', index=True)

In [102]:
distance_matrix.index

Index(['12800358', '12800354', '12800352', '12800348', '12800347'], dtype='object', name='SAMSCODE')